In [ ]:
!rm -rf graph_text
!git clone https://github.com/mikhmakarov/graph_text.git

Cloning into 'graph_text'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (408/408), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 408 (delta 236), reused 292 (delta 127), pack-reused 0
Receiving objects: 100% (408/408), 6.23 MiB | 25.60 MiB/s, done.
Resolving deltas: 100% (236/236), done.


In [ ]:
!pip install sentence_transformers

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import sys

sys.path.append("graph_text")


from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# from models import TADW, TriDnr, DeepWalk, Node2Vec, Hope
from text_transformers import Ernie
from datasets import Cora, CiteseerM10, Dblp

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from collections import defaultdict

from task import Task

Going to train on cuda


In [ ]:
datasets = [
    ('Cora', Cora),
   # ('CiteseerM10', CiteseerM10),
  #   ('DBLP', Dblp)
]

test_ratios = [0.5, 0.7, 0.9, 0.95]


tasks = [
    ('Ernie', lambda ds: Task(ds, test_ratios, lambda: Ernie(), None, d=None, labels=False)),
    # ('BOW', lambda ds: Task(ds, test_ratios, lambda: BOW(), None, d=None, labels=False)),
    # ('TFIDF', lambda ds: Task(ds, test_ratios, lambda: TFIDF(), None, d=None, labels=False)),
    # ('LDA', lambda ds: Task(ds, test_ratios, lambda: LDA(), None, d=None, labels=False)),
    # ('SBERT pretrained', lambda ds: Task(ds, test_ratios, lambda: SBert(train=False, d=300), None, d=None, labels=False)),
    # ('W2V pretrained (d=300)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=False, d=300), None, d=None, labels=False)),
    # ('W2V (d=300)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=300), None, d=None, labels=False)),
    # ('W2V (d=64)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), None, d=None, labels=False)),
    # ('Doc2Vec pretrained (d=300)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=False, d=300), None, d=None, labels=False)),
    # ('Doc2Vec (d=300)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=True, d=300), None, d=None, labels=False)),
    # ('Doc2Vec (d=64)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=True, d=64), None, d=None, labels=False)),
    # ('Sent2Vec pretrained (d=600)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=False, d=600), None, d=None, labels=False)),
    # ('Sent2Vec (d=600)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=600), None, d=None, labels=False)),
    # ('Sent2Vec (d=64)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), None, d=None, labels=False)),
    # ('DeepWalk (d=100)', lambda ds: Task(ds, test_ratios, None, DeepWalk, d=100, labels=False)),
    # ('Node2Vec (d=100)', lambda ds: Task(ds, test_ratios, None, Node2Vec, d=100, labels=False)),
    # ('Hope (d=100)', lambda ds: Task(ds, test_ratios, None, Hope, d=100, labels=False)),
    # ('TADW - BOW', lambda ds: Task(ds, test_ratios, BOW, TADW, d=160, labels=False)),
    # ('TADW - TFIDF', lambda ds: Task(ds, test_ratios, TFIDF, TADW, d=160, labels=False)),
    # ('TADW - Sent2Vec', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), TADW, d=160, labels=False)),
    # ('TADW - Word2Vec', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), TADW, d=160, labels=False)),
    # ('TriDNR', lambda ds: Task(ds, test_ratios, None, TriDnr, d=160, labels=True)),
    # ('BOW:DeepWalk', lambda ds: Task(ds, test_ratios, BOW, DeepWalk, d=100,
    #                                       labels=False, concat=True)),
    # ('Word2Vec:DeepWalk', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), DeepWalk, d=100,
    #                                       labels=False, concat=True)),
    # ('Sent2Vec:DeepWalk', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), DeepWalk, d=100,
    #                                       labels=False, concat=True)),
]

In [ ]:
res = {}

for ds_name, ds_constr in tqdm(datasets, desc='datasets'):
    ds = ds_constr()
    for task_name, task_constr in tqdm(tasks, desc='Tasks'):
        task = task_constr(ds)
        task_res = task.evaluate()
        for test_ratio in task_res:
            scores = task_res[test_ratio]
            res[f'{1 - test_ratio} - {ds_name} - {task_name}'] = scores

for name, scores in res.items():
    print(name, round(np.mean(scores), 3), round(np.std(scores), 3))



Tasks:   0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

1it [00:00,  2.01it/s]

2it [00:01,  1.99it/s]

3it [00:01,  1.97it/s]

4it [00:01,  2.16it/s]

5it [00:02,  2.13it/s]

6it [00:02,  2.35it/s]

7it [00:03,  2.47it/s]

8it [00:03,  2.31it/s]

9it [00:04,  1.89it/s]

10it [00:04,  2.01it/s]

11it [00:05,  1.83it/s]

12it [00:05,  1.93it/s]

13it [00:06,  1.94it/s]

14it [00:06,  2.15it/s]

15it [00:07,  1.66it/s]

16it [00:08,  1.43it/s]

17it [00:09,  1.47it/s]

18it [00:09,  1.59it/s]

19it [00:10,  1.44it/s]

20it [00:11,  1.56it/s]

21it [00:11,  1.73it/s]

22it [00:12,  1.78it/s]

23it [00:12,  1.91it/s]

24it [00:13,  1.88it/s]

25it [00:13,  1.99it/s]

26it [00:14,  1.67it/s]

27it [00:14,  1.90it/s]

28it [00:15,  1.81it/s]

29it [00:15,  1.87it/s]

30it [00:16,  1.90it/s]

31it [00:16,  2.10it/s]

32it [00:16,  2.31it/s]

33it [00:17,  2.20it/s]

34it [00:17,  2.40it/s]

35it [00:18,  1.93it/s]

36it [00:18,  1.99it/s]

37it [00:19,  1.98it/s]

38it [00:19,  2.08

0.5 - Cora - Ernie [0.6432791728212703, 0.6654357459379616, 0.6521418020679468, 0.6617429837518464, 0.6491875923190547] 0.654357459379616 0.008144194240525978
0.30000000000000004 - Cora - Ernie [0.6054852320675106, 0.6223628691983122, 0.6118143459915611, 0.620253164556962, 0.620253164556962] 0.6160337552742614 0.006399051007638412
0.09999999999999998 - Cora - Ernie [0.5057424118129614, 0.5004101722723544, 0.5127153404429861, 0.5377358490566038, 0.5451189499589828] 0.5203445447087777 0.017804689661380733
0.050000000000000044 - Cora - Ernie [0.43179168286047415, 0.4232413525068014, 0.4224640497473766, 0.43878740769529734, 0.4741546832491255] 0.438087835211815 0.019002469231057605


In [ ]:
len([101, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0]), len([101, 102, 0, 0, 0, 0, 0, 0])

(11, 8)

In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses pytorch-transformers

     |████████████████████████████████| 184kB 9.0MB/s 


In [ ]:
import torch
import logging
from pytorch_transformers import BertModel, BertTokenizer
from pytorch_transformers import *
from typing import List

CLS_TOKEN = "[CLS]"
SEP_TOKEN = "[SEP]"

logger = logging.getLogger(__name__)

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids


def convert_examples_to_features(examples, tokenizer, max_seq_length,
                                 cls_token_at_end=False, pad_on_left=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=1, pad_token_segment_id=0,
                                 mask_padding_with_zero=True):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """
    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d" % (ex_index, len(examples)))

        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = tokens_a + [sep_token]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if tokens_b:
            tokens += tokens_b + [sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

        if cls_token_at_end:
            tokens = tokens + [cls_token]
            segment_ids = segment_ids + [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
        else:
            input_ids = input_ids + ([pad_token] * padding_length)
            input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
            segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids))
    return features

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

def select_field(features, field):
    """As the output is dic, return relevant field"""
    return [[choice[field] for choice in feature.choices_features] for feature in features]


def create_examples(_list, set_type="train"):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(_list):
        guid = "%s-%s" % (set_type, i)
        text_a = line
        # text_b = line[1]
        examples.append(
            InputExample(guid=guid, text_a=text_a))
    return examples


class BertEmbedder:
    def __init__(self,
                 pretrained_weights='bert-base-uncased',
                 tokenizer_class=BertTokenizer,
                 model_class=BertModel,
                 max_seq_len=20):
        super().__init__()
        self.pretrained_weights = pretrained_weights
        self.tokenizer_class = tokenizer_class
        self.model_class = model_class
        self.tokenizer = self.tokenizer_class.from_pretrained(pretrained_weights)
        self.model = self.model_class.from_pretrained(pretrained_weights)
        self.max_seq_len = max_seq_len
        # tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
        # model = BertModel.from_pretrained(pretrained_weights)

    def get_bert_embeddings(self,
                            raw_text: List[str]) -> torch.tensor:
        examples = create_examples(raw_text)

        features = convert_examples_to_features(
            examples, self.tokenizer, self.max_seq_len, True)

        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)

        last_hidden_states = self.model(all_input_ids)[0]  # Models outputs are now tuples
        print(last_hidden_states.size())
        return last_hidden_states



if __name__=="__main__":
    embedder = BertEmbedder()
    raw_text = ["[CLS] This is first element [SEP] continuing statement",
                "[CLS] second element of the list."]
    bert_embedding = embedder.get_bert_embeddings(raw_text)

100%|██████████| 440473133/440473133 [00:12<00:00, 34685819.44B/s]


torch.Size([2, 20, 768])
